In [ ]:
!pip install pyngrok

In [ ]:
!pip install flask_cors

In [ ]:
import os
import numpy as np
from pyngrok import ngrok
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from flask_cors import CORS
from werkzeug.utils import secure_filename
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.densenet import preprocess_input
import tensorflow as tf

In [ ]:
ngrok.set_auth_token('Api -Key - Ngrok')

app = Flask(__name__)
CORS(app)

In [ ]:
# Define a class for Brain Tumor Prediction
class BrainTumorPredictor:
    def __init__(self):
        self.classification_model = load_model('/content/classifier/brain_image_classifier1.h5')
        self.tumor_model = load_model("/content/models/tumor.h5")
        self.classification_input_size = (150, 150)
        self.tumor_input_size = (128, 128)

    def preprocess_image(self, image_path, target_size):
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        return img_array

    def predict_image_class(self, image_path):
        preprocessed_img = self.preprocess_image(image_path, self.classification_input_size)
        prediction = self.classification_model.predict(preprocessed_img)
        return prediction[0][1] > prediction[0][0]

    def predict_brain_tumor(self, image_path):
        preprocessed_img = self.preprocess_image(image_path, self.tumor_input_size)
        prediction = self.tumor_model.predict(preprocessed_img)
        predicted_score = prediction[0][0]
        return "Brain tumor detected" if predicted_score >= 0.5 else "No brain tumor detected"

# API route for predicting brain tumor
@app.route('/predict-brain-tumor', methods=['POST'])
def predict_brain_tumor_api():
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file part'})

        file = request.files['file']
        if file.filename == '':
            return jsonify({'error': 'No selected file'})

        os.makedirs('uploads', exist_ok=True)
        filename = secure_filename(file.filename)
        filepath = os.path.join('uploads', filename)
        file.save(filepath)

        is_brain_image = brain_tumor_predictor.predict_image_class(filepath)
        if not is_brain_image:
            return jsonify({'error': 'Input image does not contain a valid brain structure'})

        prediction_result = brain_tumor_predictor.predict_brain_tumor(filepath)
        return jsonify({'prediction_result': prediction_result})

    except Exception as e:
        app.logger.error(f"An error occurred: {str(e)}")
        return jsonify({'error': 'Error detecting brain tumor. Please try again.'})

In [ ]:
# Define a class for Pneumonia Prediction
class PneumoniaPredictor:
    def __init__(self):
        self.chest_image_classifier = load_model('/content/classifier/chest_image_classifier.h5')
        self.pneumonia_model = load_model("/content/models/pneumonia_detection_model_new_v2.h5")
        self.classification_input_size = (150, 150)
        self.pneumonia_input_size = (224, 224)

    def preprocess_image(self, image_path, target_size):
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        return img_array

    def predict_image_class(self, image_path):
        preprocessed_img = self.preprocess_image(image_path, self.classification_input_size)
        prediction = self.chest_image_classifier.predict(preprocessed_img)
        return prediction[0][1] > prediction[0][0]

    def predict_pneumonia(self, image_path):
        preprocessed_img = self.preprocess_image(image_path, self.pneumonia_input_size)
        predictions = self.pneumonia_model.predict(preprocessed_img)
        class_labels = ['BACTERIAL PNEUMONIA', 'NORMAL', 'VIRUS PNEUMONIA']
        predicted_class_index = np.argmax(predictions)
        predicted_class_name = class_labels[predicted_class_index]
        confidence_percentage = predictions[0][predicted_class_index] * 100
        return predicted_class_name, confidence_percentage

# API route for predicting pneumonia
@app.route('/predict-pneumonia', methods=['POST'])
def predict_pneumonia_api():
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file part'})

        file = request.files['file']
        if file.filename == '':
            return jsonify({'error': 'No selected file'})

        os.makedirs('uploads', exist_ok=True)
        filename = secure_filename(file.filename)
        filepath = os.path.join('uploads', filename)
        file.save(filepath)

        is_chest_image = pneumonia_predictor.predict_image_class(filepath)
        if not is_chest_image:
            return jsonify({'error': 'Input image is not a valid chest image'})

        predicted_class_name, confidence_percentage = pneumonia_predictor.predict_pneumonia(filepath)
        return jsonify({'predicted_class_name': predicted_class_name, 'confidence_percentage': confidence_percentage})

    except Exception as e:
        app.logger.error(f"An error occurred: {str(e)}")
        return jsonify({'predicted_class_name': 'Error', 'confidence_percentage': 0,
                        'error_message': 'Error detecting pneumonia. Please try again.'})

In [ ]:
# Define a class for Tuberculosis Prediction
class TuberculosisPredictor:
    def __init__(self):
        self.tb_model = load_model("/content/models/tb3_model.h5")
        self.tb_chest_classifier = load_model("/content/classifier/tb_chest_image_classifier1.h5")
        self.classification_input_size = (150, 150)
        self.tb_input_size = (224, 224)

    def preprocess_image(self, image_path, target_size):
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array

    def predict_chest_class(self, image_path):
        preprocessed_img = self.preprocess_image(image_path, self.classification_input_size)
        prediction = self.tb_chest_classifier.predict(preprocessed_img)
        return "Chest Image" if prediction[0][1] > prediction[0][0] else "Non-Chest Image"

    def predict_tuberculosis(self, image_path):
        image_class = self.predict_chest_class(image_path)
        if image_class == "Non-Chest Image":
            return "Error", 0, "Uploaded image is not a chest image"

        img_array = self.preprocess_image(image_path, self.tb_input_size)
        prediction = self.tb_model.predict(img_array)

        predicted_class = "Tuberculosis Detected" if prediction[0][0] > 0.5 else "Normal (No Tuberculosis Detected)"
        confidence_percentage = prediction[0][0] * 100 if predicted_class == "Tuberculosis Detected" else (1 - prediction[0][0]) * 100

        return predicted_class, confidence_percentage, ""

# API route for predicting tuberculosis
@app.route('/predict-tuberculosis', methods=['POST'])
def predict_tuberculosis_api():
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file part'})

        file = request.files['file']

        if file.filename == '':
            return jsonify({'error': 'No selected file'})

        # Create the 'uploads' directory if it does not exist
        os.makedirs('uploads', exist_ok=True)

        # Save the file
        filename = secure_filename(file.filename)
        filepath = os.path.join('uploads', filename)
        file.save(filepath)

        # Perform prediction using the TuberculosisPredictor class
        predicted_class, confidence_percentage, error_message = tb_predictor.predict_tuberculosis(filepath)

        return jsonify({'predicted_class': predicted_class, 'confidence_percentage': confidence_percentage, 'error_message': error_message})

    except Exception as e:
        app.logger.error(f"An error occurred: {str(e)}")
        return jsonify({'predicted_class': 'Error', 'confidence_percentage': 0, 'error_message': 'Error detecting tuberculosis. Please try again.'})

In [ ]:
# Define a class for Cataract Prediction
class CataractPredictor:
    def __init__(self):
        self.eye_image_classifier = load_model('/content/classifier/eye_image_classifier.h5')
        self.cataract_model = load_model("/content/models/cat_model.h5")
        self.eye_input_size = (150, 150)
        self.cataract_image_size = (224, 224)

    def preprocess_image(self, image_path, target_size):
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        return img_array

    def predict_eye_image_class(self, image_path):
        preprocessed_img = self.preprocess_image(image_path, self.eye_input_size)
        prediction = self.eye_image_classifier.predict(preprocessed_img)
        return prediction[0][1] > prediction[0][0]

    def predict_cataract(self, image_path):
        preprocessed_img = self.preprocess_image(image_path, self.cataract_image_size)
        prediction = self.cataract_model.predict(preprocessed_img)
        predicted_class = "Cataract Detected" if prediction[0][0] > 0.5 else "Normal"
        confidence = prediction[0][0] if predicted_class == "Cataract Detected" else 1 - prediction[0][0]
        return predicted_class, confidence

# API route for predicting cataract
@app.route('/predict-cataract', methods=['POST'])
def predict_cataract_api():
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file part'})

        file = request.files['file']
        if file.filename == '':
            return jsonify({'error': 'No selected file'})

        os.makedirs('uploads', exist_ok=True)
        filename = secure_filename(file.filename)
        filepath = os.path.join('uploads', filename)
        file.save(filepath)

        is_eye_image = cataract_predictor.predict_eye_image_class(filepath)
        if not is_eye_image:
            return jsonify({'error': 'Uploaded image is not an eye image'})

        predicted_class, confidence = cataract_predictor.predict_cataract(filepath)
        confidence = float(confidence)
        return jsonify({'predicted_class': predicted_class, 'confidence': confidence})

    except Exception as e:
        app.logger.error(f"An error occurred: {str(e)}")
        return jsonify({'error': 'Error detecting cataract. Please try again.'})

In [ ]:
# Define a class for Bone Fracture Prediction
class BoneFracturePredictor:
    def __init__(self):
        self.model_elbow_frac = tf.keras.models.load_model("/content/models/bone-model/ResNet50_Elbow_frac.h5")
        self.model_hand_frac = tf.keras.models.load_model("/content/models/bone-model/ResNet50_Hand_frac.h5")
        self.model_shoulder_frac = tf.keras.models.load_model("/content/models/bone-model/ResNet50_Shoulder_frac.h5")
        self.model_parts = tf.keras.models.load_model("/content/models/bone-model/ResNet50_BodyParts.h5")
        self.model_bone_classifier = load_model('/content/classifier/bone_image_classifier3.h5')
        self.classification_input_size = (150, 150)
        self.fracture_input_size = (224, 224)
        self.categories_parts = ["Elbow", "Hand", "Shoulder"]
        self.categories_fracture = ['normal', 'fractured']

    def preprocess_image(self, image_path, target_size):
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        return img_array

    def predict_image_class(self, image_path):
        preprocessed_img = self.preprocess_image(image_path, self.classification_input_size)
        prediction = self.model_bone_classifier.predict(preprocessed_img)
        return "Bone Image" if prediction[0][1] > prediction[0][0] else "Non-Bone Image"

    def predict(self, img, model="Parts"):
        if model == 'Parts':
            chosen_model = self.model_parts
        else:
            if model == 'Elbow':
                chosen_model = self.model_elbow_frac
            elif model == 'Hand':
                chosen_model = self.model_hand_frac
            elif model == 'Shoulder':
                chosen_model = self.model_shoulder_frac

        temp_img = load_img(img, target_size=self.fracture_input_size)
        x = img_to_array(temp_img)
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])
        prediction = np.argmax(chosen_model.predict(images), axis=1)

        if model == 'Parts':
            prediction_str = self.categories_parts[prediction.item()]
        else:
            prediction_str = self.categories_fracture[prediction.item()]

        return prediction_str

  # API route for predicting bone fracture
@app.route('/predict-fracture', methods=['POST'])
def predict_fracture_api():
    try:
        if 'file' not in request.files:
            return jsonify({"error": "No file part"})
        file = request.files['file']
        if file.filename == '':
            return jsonify({"error": "No selected file"})

        os.makedirs('uploads', exist_ok=True)
        filename = secure_filename(file.filename)
        filepath = os.path.join('uploads', filename)
        file.save(filepath)

        image_class = bone_fracture_predictor.predict_image_class(filepath)
        if image_class == "Non-Bone Image":
            return jsonify({"error": "Uploaded image is not a valid bone image."})

        body_part = bone_fracture_predictor.predict(filepath, model="Parts")
        fracture_status = bone_fracture_predictor.predict(filepath, model=body_part)

        return jsonify({
            "predicted_body_part": body_part,
            "predicted_fracture_status": fracture_status
        })

    except Exception as e:
        app.logger.error(f"An error occurred: {str(e)}")
        return jsonify({"error": "Error detecting bone fracture. Please try again."})

In [ ]:
# Initialize predictors
brain_tumor_predictor = BrainTumorPredictor()
cataract_predictor = CataractPredictor()
pneumonia_predictor = PneumoniaPredictor()
tb_predictor = TuberculosisPredictor()
bone_fracture_predictor = BoneFracturePredictor()

In [ ]:
if __name__ == "__main__":

    port = 5000
    public_url = ngrok.connect(port)
    print(' * ngrok tunnel "{}"'.format(public_url))

    # Update any base URLs or webhooks to use the public ngrok URL
    app.config['BASE_URL'] = public_url  # You can use this base URL in your frontend

    # Start the Flask app
    app.run()

 * ngrok tunnel "NgrokTunnel: "https://6a2a-34-122-19-16.ngrok-free.app" -> "http://localhost:5000""
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 [==============================] - 1s 1s/step


INFO:werkzeug:127.0.0.1 - - [25/May/2024 11:40:05] "POST /predict-fracture HTTP/1.1" 200 -


1/1 [==============================] - 0s 126ms/step


...

INFO:werkzeug:127.0.0.1 - - [27/May/2024 08:53:07] "POST /predict-cataract HTTP/1.1" 200 -


1/1 [==============================] - 0s 117ms/step


INFO:werkzeug:127.0.0.1 - - [27/May/2024 08:53:47] "POST /predict-cataract HTTP/1.1" 200 -


1/1 [==============================] - 0s 136ms/step


INFO:werkzeug:127.0.0.1 - - [27/May/2024 08:54:10] "POST /predict-cataract HTTP/1.1" 200 -
